In [20]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sympy import *
from sympy.stats import *
import math

In [21]:
parameters = pd.read_csv('parameters.csv')
mean_parameters = parameters.iloc[0::2]
mean_parameters=mean_parameters.reset_index().drop('index',axis=1)
mean_parameters

,Rv,distribution,parameters,Zone 1,Zone 2,Zone 3,Zone 4,Zone 5,Zone 6,Zone 7
0,err,N,mean,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000
1,su,N,mean,46.857143,56.239161,44.597203,51.547153,37.073526,49.830569,54.00969
2,a,N,mean,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.00000
3,gamma,N,mean,18.400000,18.340000,17.300000,18.780000,18.720000,19.640000,16.88000
4,R,N,mean,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.00000
5,eta,N,mean,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.10000
6,q,N,mean,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000


In [22]:
std_parameters= parameters.iloc[1::2].reset_index().drop('index',axis=1)
std_parameters

,Rv,distribution,parameters,Zone 1,Zone 2,Zone 3,Zone 4,Zone 5,Zone 6,Zone 7
0,err,N,std,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
1,su,N,std,8.468764,7.557970,9.732100,5.190861,6.058854,9.153118,11.091138
2,a,N,std,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
3,gamma,N,std,1.019804,2.039117,2.732215,1.976613,1.395349,2.169793,2.591718
4,R,N,std,1.600000,1.600000,1.600000,1.600000,1.600000,1.600000,1.600000
5,eta,N,std,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000
6,q,N,std,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


Now we are going to calculate mean and standard deviation of $F_s$. Here, we are considering all the random variables are independent from each other and all of them are Normally distributed.

In [23]:
df = pd.DataFrame(columns=mean_parameters.iloc[0:1,0:10].columns)
df=df.drop('distribution',axis=1)
zones = df.drop(['Rv','parameters'],axis=1).columns
zones

Index(['Zone 1', 'Zone 2', 'Zone 3', 'Zone 4', 'Zone 5', 'Zone 6', 'Zone 7'], dtype='object')

In [24]:
z1=mean_parameters.loc[:,zones[0]]
z2=std_parameters.loc[:,zones[0]]
err=Normal('\epsilon',z1[0],z2[0])
su=Normal('s_u',z1[1],z2[1])
a=Normal(r'\alpha',math.radians(z1[2]),math.radians(z2[2]))
gamma=Normal('\gamma',z1[3],z2[3])
R=Normal('R',z1[4],z2[4])
n=Normal('\eta',z1[5],z2[5])
q = Normal('q',z1[6],z2[6])
Fs=(err*su*a)/(1/4*gamma*R*(a-sin(a))*sin(a)+n*q*sin(a))
Fs

\alpha*\epsilon*s_u/(0.25*(-sin(\alpha) + \alpha)*sin(\alpha)*R*\gamma + sin(\alpha)*\eta*q)

In [25]:
fs = ['Fs','mean']
for zone in zones:
    z1=mean_parameters.loc[:,zone]
    meanFs=Fs.subs([(err,z1[0]),(su,z1[1]),(a,math.radians(z1[2])),(gamma,z1[3]),(R,z1[4]),(n,z1[5]),(q,z1[6])])
    fs.append(meanFs)
# Convert the list to a DataFrame and set the column names to match your existing DataFrame
new_row_df = pd.DataFrame([fs], columns=df.columns)

# Append the new row to the existing DataFrame
df = pd.concat([df, new_row_df], ignore_index=True)
df

,Rv,parameters,Zone 1,Zone 2,Zone 3,Zone 4,Zone 5,Zone 6,Zone 7
0,Fs,mean,7.38993601908864,8.89481512463578,7.41920630328050,7.98618656180799,5.75983576583756,7.42383224122838,9.17721802835119


In [26]:
Fs

\alpha*\epsilon*s_u/(0.25*(-sin(\alpha) + \alpha)*sin(\alpha)*R*\gamma + sin(\alpha)*\eta*q)

In [27]:
fs = ['Fs','std']
for zone in zones:
    z1=mean_parameters.loc[:,zone]
    z2=std_parameters.loc[:,zone]
    err=Normal('\epsilon',z1[0],z2[0])
    su=Normal('s_u',z1[1],z2[1])
    a=Normal(r'\alpha',math.radians(z1[2]),math.radians(z2[2]))
    gamma=Normal('\gamma',z1[3],z2[3])
    R=Normal('R',z1[4],z2[4])
    n=Normal('\eta',z1[5],z2[5])
    q = Normal('q',z1[6],z2[6])
    Fs=(err*su*a)/(1/4*gamma*R*(a-sin(a))*sin(a)+n*q*sin(a))
    std = sqrt((sqrt(variance(err))*diff(Fs,err))**2 + (sqrt(variance(su))*diff(Fs,su))**2 + (sqrt(variance(a))*diff(Fs,a))**2 + (sqrt(variance(gamma))*diff(Fs,gamma))**2+ (sqrt(variance(R))*diff(Fs,R))**2+ (sqrt(variance(n))*diff(Fs,n))**2+ (sqrt(variance(q))*diff(Fs,q))**2)
    stdFs=std.subs([(err,z1[0]),(su,z1[1]),(a,math.radians(z1[2])),(gamma,z1[3]),(R,z1[4]),(n,z1[5]),(q,z1[6])])
    fs.append(stdFs)
# Convert the list to a DataFrame and set the column names to match your existing DataFrame
new_row_df = pd.DataFrame([fs], columns=df.columns)

# Append the new row to the existing DataFrame
df = pd.concat([df, new_row_df], ignore_index=True)
df

,Rv,parameters,Zone 1,Zone 2,Zone 3,Zone 4,Zone 5,Zone 6,Zone 7
0,Fs,mean,7.38993601908864,8.89481512463578,7.41920630328050,7.98618656180799,5.75983576583756,7.42383224122838,9.17721802835119
1,Fs,std,3.10897453492980,3.65965848691857,3.36586733016770,3.20772823757525,2.39970394820932,3.21493820506062,4.08634383784991


Now we are going to make a performance function for FOSM analysis.

In [28]:
g_x = (err*su*a)/(1/4*gamma*R*(a-sin(a))*sin(a)+n*q*sin(a))-1
g_x

-1 + \alpha*\epsilon*s_u/(0.25*(-sin(\alpha) + \alpha)*sin(\alpha)*R*\gamma + sin(\alpha)*\eta*q)

Now we are going to calculate the mean, standard deviation, and reliability index of the performance function for every zone

In [29]:
dfg_x = pd.DataFrame(columns=df.columns)
dfg_x

,Rv,parameters,Zone 1,Zone 2,Zone 3,Zone 4,Zone 5,Zone 6,Zone 7


mean of performance function

In [30]:
fs = ['g_x','mean']
for zone in zones:
    z1=mean_parameters.loc[:,zone]
    meanFs=g_x.subs([(err,z1[0]),(su,z1[1]),(a,math.radians(z1[2])),(gamma,z1[3]),(R,z1[4]),(n,z1[5]),(q,z1[6])])
    fs.append(meanFs)
# Convert the list to a DataFrame and set the column names to match your existing DataFrame
new_row_df = pd.DataFrame([fs], columns=dfg_x.columns)

# Append the new row to the existing DataFrame
dfg_x = pd.concat([dfg_x, new_row_df], ignore_index=True)
dfg_x

,Rv,parameters,Zone 1,Zone 2,Zone 3,Zone 4,Zone 5,Zone 6,Zone 7
0,g_x,mean,6.38993601908864,7.89481512463578,6.41920630328050,6.98618656180799,4.75983576583756,6.42383224122838,8.17721802835119


standard deviation of performance function

In [31]:
fs = ['g_x','std']
for zone in zones:
    z1=mean_parameters.loc[:,zone]
    z2=std_parameters.loc[:,zone]
    err=Normal('\epsilon',z1[0],z2[0])
    su=Normal('s_u',z1[1],z2[1])
    a=Normal(r'\alpha',math.radians(z1[2]),math.radians(z2[2]))
    gamma=Normal('\gamma',z1[3],z2[3])
    R=Normal('R',z1[4],z2[4])
    n=Normal('\eta',z1[5],z2[5])
    q = Normal('q',z1[6],z2[6])
    # defining g_x again because I want variance for different zones
    g_x=(err*su*a)/(1/4*gamma*R*(a-sin(a))*sin(a)+n*q*sin(a))
    std = sqrt((sqrt(variance(err))*diff(g_x,err))**2 + (sqrt(variance(su))*diff(g_x,su))**2 + (sqrt(variance(a))*diff(g_x,a))**2 + (sqrt(variance(gamma))*diff(g_x,gamma))**2+ (sqrt(variance(R))*diff(g_x,R))**2+ (sqrt(variance(n))*diff(g_x,n))**2+ (sqrt(variance(q))*diff(g_x,q))**2)
    stdg_x=std.subs([(err,z1[0]),(su,z1[1]),(a,math.radians(z1[2])),(gamma,z1[3]),(R,z1[4]),(n,z1[5]),(q,z1[6])])
    fs.append(stdg_x)
# Convert the list to a DataFrame and set the column names to match your existing DataFrame
new_row_df = pd.DataFrame([fs], columns=dfg_x.columns)

# Append the new row to the existing DataFrame
dfg_x = pd.concat([dfg_x, new_row_df], ignore_index=True)
dfg_x

,Rv,parameters,Zone 1,Zone 2,Zone 3,Zone 4,Zone 5,Zone 6,Zone 7
0,g_x,mean,6.38993601908864,7.89481512463578,6.41920630328050,6.98618656180799,4.75983576583756,6.42383224122838,8.17721802835119
1,g_x,std,3.10897453492980,3.65965848691857,3.36586733016770,3.20772823757525,2.39970394820932,3.21493820506062,4.08634383784991


In [32]:
fs = ['β','reliability index',1,2,3,4,5,6,7]
new_row_df = pd.DataFrame([fs], columns=dfg_x.columns)
dfg_x = pd.concat([dfg_x, new_row_df], ignore_index=True)
dfg_x

,Rv,parameters,Zone 1,Zone 2,Zone 3,Zone 4,Zone 5,Zone 6,Zone 7
0,g_x,mean,6.38993601908864,7.89481512463578,6.41920630328050,6.98618656180799,4.75983576583756,6.42383224122838,8.17721802835119
1,g_x,std,3.10897453492980,3.65965848691857,3.36586733016770,3.20772823757525,2.39970394820932,3.21493820506062,4.08634383784991
2,β,reliability index,1,2,3,4,5,6,7


In [33]:
dfg_x.iloc[2,2:9]=dfg_x.iloc[0,2:9]/dfg_x.iloc[1,2:9]
dfg_x

,Rv,parameters,Zone 1,Zone 2,Zone 3,Zone 4,Zone 5,Zone 6,Zone 7
0,g_x,mean,6.38993601908864,7.89481512463578,6.41920630328050,6.98618656180799,4.75983576583756,6.42383224122838,8.17721802835119
1,g_x,std,3.10897453492980,3.65965848691857,3.36586733016770,3.20772823757525,2.39970394820932,3.21493820506062,4.08634383784991
2,β,reliability index,2.05531951043559,2.15725460527417,1.90714774933232,2.17792345373027,1.98350957808333,1.99811997353997,2.00110865674333


In [34]:
fs = ['Pf','failure Probability',1,2,3,4,5,6,7]
new_row_df = pd.DataFrame([fs], columns=dfg_x.columns)
dfg_x = pd.concat([dfg_x, new_row_df], ignore_index=True)
dfg_x

,Rv,parameters,Zone 1,Zone 2,Zone 3,Zone 4,Zone 5,Zone 6,Zone 7
0,g_x,mean,6.38993601908864,7.89481512463578,6.41920630328050,6.98618656180799,4.75983576583756,6.42383224122838,8.17721802835119
1,g_x,std,3.10897453492980,3.65965848691857,3.36586733016770,3.20772823757525,2.39970394820932,3.21493820506062,4.08634383784991
2,β,reliability index,2.05531951043559,2.15725460527417,1.90714774933232,2.17792345373027,1.98350957808333,1.99811997353997,2.00110865674333
3,Pf,failure Probability,1,2,3,4,5,6,7


In [40]:
dfg_x.iloc[3,2:9]=dfg_x.iloc[2,2:9].apply(lambda x:stats.norm.cdf(-float(x),loc=0,scale=1))
dfg_x

,Rv,parameters,Zone 1,Zone 2,Zone 3,Zone 4,Zone 5,Zone 6,Zone 7
0,g_x,mean,6.38993601908864,7.89481512463578,6.41920630328050,6.98618656180799,4.75983576583756,6.42383224122838,8.17721802835119
1,g_x,std,3.10897453492980,3.65965848691857,3.36586733016770,3.20772823757525,2.39970394820932,3.21493820506062,4.08634383784991
2,β,reliability index,2.05531951043559,2.15725460527417,1.90714774933232,2.17792345373027,1.98350957808333,1.99811997353997,2.00110865674333
3,Pf,failure Probability,0.019924,0.015493,0.028251,0.014706,0.023655,0.022852,0.02269
